In [5]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
from sklearn.preprocessing import StandardScaler
import pandas
from feature_function import select_features
from data_functions import his_player_defense_data, current_player_defense_data, build_data_path, his_usage_team
import pandas as pd
import numpy as np
from IPython.display import display
import sys
from datetime import datetime
import os




import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import linregress


def prediction(player_names: dict, date_list: list, stats_path: dict, player_base_path, defense_base_path, schedule_base_path ,selected_feature_target, prediction_target):
    """
    Predicts future data for NBA players using historical data and rolling averages.

    Parameters:
        player_names (dict): Dictionary mapping player names to their respective teams.
        date_list (list): List of dates to consider for analysis.
        usage_path (str): Path to player usage data.
        player_base_path (str): Path to player statistics data.
        defense_base_path (str): Path to team defense statistics.
        schedule_base_path (str): Path to team schedules.
        selected_feature_target (str): Target feature for selecting model input features.
        prediction_target (str): The feature to be predicted (e.g., FGA).

    Returns:
        pd.DataFrame: A DataFrame containing predicted values and RMSE for each player.
    """
    fga_prediction_data, df_defense = his_usage_team(player_names, date_list, stats_path, player_base_path, defense_base_path)
    fga_prediction_results = {}

    # Select the best features for each player
    feature_dic = select_features(player_names, date_list, stats_path, player_base_path, defense_base_path, selected_feature_target)

    for player, team in player_names.items():
        # Load schedule data for the player's team
        schedule_path = build_data_path(schedule_base_path, schedule_team=team)
        schedule_df = pd.read_csv(schedule_path)

        # Retrieve player-specific prediction data
        df = fga_prediction_data[player]


        features = feature_dic[player] 

        # some players can't connect to the his_usage datatframe well I don't why
        features = [feature for feature in features if feature in df.columns]
        
        if not features:
            print(f"No valid features found for {player} for predicting {prediction_target}")
            continue



        target = prediction_target

        # print(f"Available columns for {player}: {df.columns.tolist()}")
        # print(f"Selected features for {player}: {features}")

        # Split data into training and testing sets based on a timestamp
        timestamp = int(pd.Timestamp('2025-03-10').timestamp())
        train_data = df[df['Date_in_Seconds'] < timestamp]
        test_data = df[df['Date_in_Seconds'] >= timestamp]

        X_train = train_data[features].fillna(0)
        y_train = train_data[target].fillna(0)
        X_test = test_data[features].fillna(0)
        y_test = test_data[target].fillna(0) 

        # display(X_train.head(1))
        # print(f"X_train shape: {X_train.shape}")
        # print(f"y_train shape: {y_train.shape}")
        # print(X_train.head())  # View first few rows
        # print(y_train.head())  # View first few rows

        # print(features)  # Check selected feature names
        # print(X_train.columns)  # See available columns


        if not features:  # If feature_columns is an empty list
            print(f"Skipping training: No selected {target} features.")
            sys.exit(1) 



        # Train linear regression model
        model = LinearRegression()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Calculate error metrics
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)

        print(f"{player}, MAE: {mae}, RMSE: {rmse}")

        # Define feature exclusion lists
        exclude_features = ['RANK', 'OffRtg', 'W', 'L', 'DefRtg', 'NetRtg', 'AST%', 'AST/TO', 'ASTRatio',
                            'OREB%', 'DREB%', 'REB%', 'TOV%', 'eFG%', 'TS%', 'PACE', 'POSS', 'TEAM', 'PIE']
        exclude_features_schedule = ['home_away', 'schedule_team', 'DATE', 'location', 'season_defense']

        # Convert schedule dates to timestamp format
        schedule_df['Date_in_Seconds'] = pd.to_datetime(schedule_df['DATE']).astype('int64') // 10**9
        schedule_df['home_away'] = schedule_df['location'].apply(lambda x: 1 if x == 'away' else 0)

        # Extract defensive stats for the scheduled team
        last_season = df_defense["season_defense"].iloc[-1]
        df_for_schedule = df_defense.loc[df_defense["season_defense"] == last_season, exclude_features]
        first_team = schedule_df['schedule_team'].iloc[0]
        schedule_team_result = schedule_df.loc[schedule_df['schedule_team'] == first_team, 'schedule_team'].values[0]
        schedule_values = {feature: df_for_schedule.loc[df_for_schedule['TEAM'] == schedule_team_result, feature].values[0] 
                           for feature in exclude_features if feature in df_for_schedule.columns}
        
 
        # print(schedule_values)  # Debugging: Ensure schedule values are correctly retrieved

        # Extract rolling average features
        rolling_features = [col for col in features if col not in exclude_features]
        for col in rolling_features:
            df[f'{col}'] = df[col].rolling(window=20).mean().fillna(0).astype(int)
        df_last_rolling = df.iloc[[-1]][[f'{col}' for col in rolling_features]].reset_index(drop=True)

        # Update rolling feature set with scheduled team values
        for value in features:
            if value in exclude_features:
                df_last_rolling[value] = schedule_values.get(value)
        for value in features:
            if value in exclude_features_schedule:
                df_last_rolling[value] = schedule_df[value].iloc[0]

        # Ensure feature order is consistent with model input
        df_last_rolling = df_last_rolling.reindex(columns=features)
        X_future = df_last_rolling

        # display(X_future.head(10))
        # Step 1: Calculate mean and standard deviation of the target variable (PTS, REB, etc.)
        mean_target = y_train.mean()
        std_target = y_train.std()

        # Step 2: Define reasonable bounds (e.g., within 3 standard deviations)
        lower_bound = mean_target - 3 * std_target
        upper_bound = mean_target + 3 * std_target

        

        # future predictions happens here
        future_predictions = model.predict(X_future).astype('int')

        future_predictions = np.clip(future_predictions, lower_bound, upper_bound).astype('int')



        # creating rolling mean average
        df[f"Rolling_Mean_{target}"] = df[target].rolling(window=20).mean()
        df[f"Rolling_Std_{target}"] = df[target].rolling(window=20).std()
        df[f"Rolling_CV_{target}"] = df[f"Rolling_Std_{target}"] / df[f"Rolling_Mean_{target}"]

        rounded_future_prediction = abs(future_predictions[0])

        # print(player)
        # display(X_future.head(10))
        # print(player, target, rounded_future_prediction)
        # display(df[[f"Rolling_CV_{target}"]].tail(1))

        if pd.isna(df[f"Rolling_CV_{target}"].iloc[-1]) or np.isinf(df[f"Rolling_CV_{target}"].iloc[-1]):
            df.loc[df.index[-1], f"Rolling_CV_{target}"] = 0

        rolling_cv = df[f"Rolling_CV_{target}"].iloc[-1]
        highest_cv_seen = df[f"Rolling_CV_{target}"].max()

        cv_fluctuate = rolling_cv * rounded_future_prediction

        if cv_fluctuate > rounded_future_prediction:
            cv_low_prediction = abs(cv_fluctuate - rounded_future_prediction)

        cv_low_prediction = abs(rounded_future_prediction- cv_fluctuate)
        cv_high_prediction = rounded_future_prediction + cv_fluctuate

        player_prediction = f"{cv_low_prediction.astype('int')} - {rounded_future_prediction} - {cv_high_prediction.astype('int')}"

        

        if rolling_cv > 1:
            confidence_score = max(0, 1 - (rolling_cv / highest_cv_seen))
        else:
            confidence_score = 1 - rolling_cv  # More stability → Higher confidence

        confidence_score_percentage = round(confidence_score * 100, 2)


        lower_bound, upper_bound = cv_low_prediction, rounded_future_prediction




        # Get last 10 games
        recent_games = df[target].tail(10)

        


        # Fit a linear regression (x = game number, y = points)
        x = np.arange(1, len(recent_games) + 1)
        slope, intercept, r_value, p_value, std_err = linregress(x, recent_games)

        long_term_cv = df["PTS"].rolling(10).std() / df["PTS"].rolling(10).mean()

        # Set dynamic base threshold (scaled by long-term CV)
        base_threshold = max(0.2, min(0.6, 0.3 + 0.2 * long_term_cv.iloc[-1]))

        # Compute dynamic middle threshold (adjusted for rolling CV)
        middle_threshold = max(0.2, min(0.8, base_threshold * (1 + rolling_cv)))
        
        # Check if the slope is close to zero (i.e., in the middle)
        if -middle_threshold <= slope <= middle_threshold:
            trend_status = "stable"
        elif slope > 0:
            trend_status = "trending up"
        else:
            trend_status = "trending down"


        ##### this is for safebet column ############
        import math
        # Step 1: Calculate the midpoint of the range
        midpoint = (lower_bound + upper_bound) / 2
        midpoint = math.floor(midpoint)
    
        
        # Step 2: Adjust the prediction based on confidence score
        if confidence_score_percentage > 60:
            # High confidence - stick closer to midpoint
            if trend_status == "trending up":
                # Trending up - lean towards the higher end
                exact_point = round(midpoint)
            elif trend_status == "trending down":
                # Trending down - lean towards the lower end
                exact_point = lower_bound
            else:
                # Stable - pick midpoint or the closest round number
                exact_point = round(midpoint)
        else:
            # Low confidence - lean more conservatively towards the edges
            if trend_status == "trending up":
                # Trending up - lean towards the higher end
                exact_point = round(midpoint + 1)  # Slight bias to upper end
            elif trend_status == "trending down":
                # Trending down - lean towards the lower end
                exact_point = round(midpoint - 1)  # Slight bias to lower end
            else:
                # Stable - pick midpoint but be cautious (lean lower)
                exact_point = round(midpoint - 1)

        exact_point = int(exact_point)

        if exact_point == -1:
            exact_point = 0

        if target == 'FGA':
            target = 'PTS'

        # trend_df = df[target].tail(10).tolist()
        trend_df = df[target].tail(10).tolist()[::-1]



        fga_prediction_results[player] = [team,player_prediction,confidence_score_percentage,trend_df]

        
        
        df_results = pd.DataFrame.from_dict(fga_prediction_results, orient='index', columns=['team',target,f'confidence_level_{target}' ,f'recentgames_{target}'])
        # Reset index and rename it properly
        df_results.reset_index(inplace=True)
        df_results.rename(columns={'index': 'Player'}, inplace=True)
        today_date = datetime.today().strftime('%Y-%m-%d')
        #create a directory if one doesn't exist
        directory = f'prediction_output/{target}_outputs'
        os.makedirs(directory, exist_ok=True)    
        df_results.to_csv(f'{directory}/{target}_output_{today_date}.csv', index=False)
        # display(df_results)

    return df_results

In [6]:
# from prediction import prediction



players_teams = {
    "Dyson Daniels": "ATL",
    "Zaccharie Risacher": "ATL",
    "Trae Young": "ATL",
    "Mouhamed Gueye": "ATL",
    "Onyeka Okongwu": "ATL",
    "Tyrese Martin": "BKN",
    "Ziaire Williams": "BKN",
    "Keon Johnson": "BKN",
    "Cameron Johnson": "BKN",
    "Nic Claxton": "BKN",
    "Jamison Battle": "TOR",
    "RJ Barrett": "TOR",
    "Jamal Shead": "TOR",
    "Scottie Barnes": "TOR",
    "Jakob Poeltl": "TOR",
    "Shaedon Sharpe": "POR",
    "Toumani Camara": "POR",
    "Anfernee Simons": "POR",
    "Deni Avdija": "POR",
    "Donovan Clingan": "POR",
    "Josh Green": "CHA",
    "DaQuan Jeffries": "CHA",
    "LaMelo Ball": "CHA",
    "Miles Bridges": "CHA",
    "Mark Williams": "CHA",
    "Bogdan Bogdanovic": "LAC",
    "Kawhi Leonard": "LAC",
    "James Harden": "LAC",
    "Nicolas Batum": "LAC",
    "Ivica Zubac": "LAC",
    "Collin Sexton": "UTA",
    "Cody Williams": "UTA",
    "Isaiah Collier": "UTA",
    "Lauri Markkanen": "UTA",
    "Kyle Filipowski": "UTA",
    "Anthony Edwards": "MIN",
    "Jaden McDaniels": "MIN",
    "Donte DiVincenzo": "MIN",
    "Julius Randle": "MIN",
    "Rudy Gobert": "MIN",
    "Cason Wallace": "OKC",
    "Aaron Wiggins": "OKC",
    "Shai Gilgeous-Alexander": "OKC",
    "Kenrich Williams": "OKC",
    "Chet Holmgren": "OKC",
    "Taurean Prince": "MIL",
    "Kyle Kuzma": "MIL",
    "Damian Lillard": "MIL",
    "Giannis Antetokounmpo": "MIL",
    "Brook Lopez": "MIL"
}





# remember to run mover
# remember to check schedule csv
import pandas as pd
date_list = ["2022-23","2023-24","2024-25"]
stats_path = {
    'usage_path':'D:/nba_usage_csv_historic/usage_csv_{date}/{date}_content.csv',
    'catch_shoot':"D:/nba_tracking_data_csv/nba_csv_{date}/catch_shoot_content.csv",
    'drives':"D:/nba_tracking_data_csv/nba_csv_{date}/drives_content.csv",
    'elbow_touches':"D:/nba_tracking_data_csv/nba_csv_{date}/elbow_touch_content.csv",
    'paint_touches':"D:/nba_tracking_data_csv/nba_csv_{date}/paint_touch_content.csv",
    'passing':"D:/nba_tracking_data_csv/nba_csv_{date}/passing_content.csv",
    'pullup':"D:/nba_tracking_data_csv/nba_csv_{date}/pullup_content.csv",
    'shooting_efficiency':"D:/nba_tracking_data_csv/nba_csv_{date}/shooting_efficiency_content.csv",
    'touches':"D:/nba_tracking_data_csv/nba_csv_{date}/touches_content.csv",
    'tracking_post_ups_content':"D:/nba_tracking_data_csv/nba_csv_{date}/tracking_post_ups_content.csv"
}
schedule_base_path = "D:/nba_scheduled_csv/schedule_csv_2025/{schedule_team}_schedule_content.csv"
player_base_path = "D:/nba_player_csv_historic/season_{date}/all_quarters/{player}_content.csv"
defense_base_path = "D:/nba_defense_history_csv/defense_csv_{date}/all_quarter_defense_content.csv"

results_reb = prediction(players_teams, date_list, stats_path, player_base_path, defense_base_path, schedule_base_path,'REB','REB')
results_ast = prediction(players_teams, date_list, stats_path, player_base_path, defense_base_path, schedule_base_path,'AST','AST')
results_pts = prediction(players_teams, date_list, stats_path, player_base_path, defense_base_path, schedule_base_path,'PTS','PTS')
results_3pm = prediction(players_teams, date_list, stats_path, player_base_path, defense_base_path, schedule_base_path,'3PM','3PM')
results_pts = results_pts.rename(columns={'FGA': 'PTS'})



# display(results_reb)
# display(results_ast)
# display(results_pts)
# display(results_3pm)








Player data missing for Zaccharie Risacher on 2022-23, skipping.
Skipping merge for Zaccharie Risacher on 2022-23 due to missing data.
defense data skipping defense merge
Player data missing for Zaccharie Risacher on 2023-24, skipping.
Skipping merge for Zaccharie Risacher on 2023-24 due to missing data.
defense data skipping defense merge
Player data missing for Mouhamed Gueye on 2022-23, skipping.
Skipping merge for Mouhamed Gueye on 2022-23 due to missing data.
defense data skipping defense merge
Player data missing for Tyrese Martin on 2023-24, skipping.
Skipping merge for Tyrese Martin on 2023-24 due to missing data.
defense data skipping defense merge
Player data missing for Jamison Battle on 2022-23, skipping.
Skipping merge for Jamison Battle on 2022-23 due to missing data.
defense data skipping defense merge
Player data missing for Jamison Battle on 2023-24, skipping.
Skipping merge for Jamison Battle on 2023-24 due to missing data.
defense data skipping defense merge
Player d

c:\Users\mandy\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.575e-03, tolerance: 2.055e-03
  model = cd_fast.enet_coordinate_descent(
c:\Users\mandy\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.194e-02, tolerance: 5.888e-02
  model = cd_fast.enet_coordinate_descent(
c:\Users\mandy\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the sca

Dyson Daniels, MAE: 1.7422700268726359, RMSE: 2.2981819790667783
Zaccharie Risacher, MAE: 0.6999675929713777, RMSE: 0.7953826756739946
Trae Young, MAE: 0.8725412745706116, RMSE: 0.9614084676670797
Mouhamed Gueye, MAE: 4.454606394441373, RMSE: 5.30444788547135
Onyeka Okongwu, MAE: 0.9996424706952496, RMSE: 1.141108889328913
Tyrese Martin, MAE: 1.795617366025186, RMSE: 2.3835437951713794
Ziaire Williams, MAE: 0.4951937132732399, RMSE: 0.5600579226482468
Keon Johnson, MAE: 0.7479907654643081, RMSE: 1.0073482176125674
Cameron Johnson, MAE: 1.8372719296859108, RMSE: 1.9074251148923913
Nic Claxton, MAE: 1.9395551079283468, RMSE: 1.990153063405249
Jamison Battle, MAE: 0.9313497207763675, RMSE: 1.1789425080403202
RJ Barrett, MAE: 0.97522004799265, RMSE: 0.9753068017921043
Jamal Shead, MAE: 3.786373642948046, RMSE: 4.92774819180182
Scottie Barnes, MAE: 1.3135253901276567, RMSE: 1.58055971927734
Jakob Poeltl, MAE: 1.9392433341368331, RMSE: 1.9392433341368331
Shaedon Sharpe, MAE: 3.68199925162347

c:\Users\mandy\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(
c:\Users\mandy\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(
c:\Users\mandy\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the sca

Dyson Daniels, MAE: 1.987571186111979e-14, RMSE: 2.0496737061651214e-14
Zaccharie Risacher, MAE: 4.366877230192283e-15, RMSE: 4.573975141006881e-15
Trae Young, MAE: 8.929186051875676e-14, RMSE: 8.933250525072583e-14
Mouhamed Gueye, MAE: 0.43419094245218837, RMSE: 0.5135368098086468
Onyeka Okongwu, MAE: 1.0363347633410603e-14, RMSE: 1.1136062169770078e-14
Tyrese Martin, MAE: 9.986456106503283e-14, RMSE: 1.1169908552326534e-13
Ziaire Williams, MAE: 2.797894194106476e-09, RMSE: 3.2179751641970706e-09
Keon Johnson, MAE: 2.870180622375477e-13, RMSE: 2.9559200908637647e-13
Cameron Johnson, MAE: 5.55629842136085e-09, RMSE: 5.661098003989629e-09
Nic Claxton, MAE: 5.204170427930421e-18, RMSE: 5.204170427930421e-18
Jamison Battle, MAE: 1.1515522609547209e-09, RMSE: 1.253362973140404e-09
RJ Barrett, MAE: 1.4432899320127035e-14, RMSE: 1.449425816734581e-14
Jamal Shead, MAE: 5.520539814362242e-10, RMSE: 5.874584116362568e-10
Scottie Barnes, MAE: 5.06609694613676e-09, RMSE: 5.282507302298243e-09
Jak

In [3]:

# display(results_reb)
# display(results_ast)
# display(results_pts)
# display(results_3pm)

df_merged = results_reb.merge(results_ast, on=['Player', 'team'], suffixes=('_reb', '_ast')) \
    .merge(results_pts, on=['Player', 'team'], suffixes=('_pts','extra_pts')) \
    .merge(results_3pm, on=['Player', 'team'], suffixes=('_pts','_3pm'))

['Player', 'REB', 'confidence_level_REB', 'middlebet_REB', 'AST', 'confidence_level_AST', 'middlebet_AST', 'PTS', 'confidence_level_PTS', 'middlebet_PTS', '3PM', 'confidence_level_3PM', 'middlebet_3PM']

# print(df_merged.columns)


# df_merged['middlebet_PTS+REB'] = 0000

# df_merged['PTS+REB'] = df_merged[['middlebet_PTS', 'middlebet_REB']].sum(axis=1)
# df_merged['confidence_level_PTS+REB']=  round(df_merged[['confidence_level_PTS','confidence_level_REB']].sum(axis=1)/2,2)
# df_merged = df_merged.rename(columns={'FGA': 'PTS'})
# display(df_merged)
# display(df_merged[['Player','confidence_level_PTS+REB']])


from parlay_picker import create_parlays

num_groups = 100  # Number of parlays you want to generate
players_per_group = 3  # Number of players in each parlay
min_confidence = 60

parlays_df, group_confidence_scores = create_parlays(df_merged, num_groups, players_per_group,min_confidence)


# Display the confidence scores for each group
print("Confidence scores for each group:")
for group, score in group_confidence_scores:
    print(f"{group}: {score:.2f}")


# Display the DataFrames for each parlay and their confidence scores
for (parlay_name, score), parlay_df in zip(group_confidence_scores, parlays_df):
    print("\n")
    print(f"{parlay_name}: total confidence score: {score:.2f}")
    display(parlay_df)
    print("\n")

Confidence scores for each group:
Parlay 1: 67.07
Parlay 2: 62.50
Parlay 3: 80.97
Parlay 4: 83.42
Parlay 5: 78.75
Parlay 6: 81.38
Parlay 7: 62.88
Parlay 8: 65.14
Parlay 9: 77.60
Parlay 10: 65.43
Parlay 11: 73.77
Parlay 12: 62.02


Parlay 1: total confidence score: 67.07


,Player,team,Stat,Stat Range,Confidence,recentgames
0,Onyeka Okongwu,ATL,REB,6 - 9 - 11,69.58,"[10, 10, 5, 16, 13, 9, 12, 13, 9, 8]"
1,Trae Young,ATL,AST,7 - 11 - 14,67.76,"[7, 12, 8, 16, 13, 15, 12, 8, 14, 13]"
2,Trae Young,ATL,PTS,15 - 25 - 34,63.88,"[17, 35, 36, 22, 28, 12, 19, 17, 11, 38]"






Parlay 2: total confidence score: 62.50


,Player,team,Stat,Stat Range,Confidence,recentgames
0,Dyson Daniels,ATL,REB,3 - 6 - 8,63.34,"[6, 5, 6, 6, 4, 9, 3, 4, 5, 11]"
1,Onyeka Okongwu,ATL,3PM,0 - 0 - 0,62.93,"[0, 1, 3, 0, 0, 0, 0, 1, 1, 2]"
2,Dyson Daniels,ATL,AST,2 - 4 - 5,61.24,"[5, 3, 9, 4, 5, 6, 7, 3, 5, 2]"






Parlay 3: total confidence score: 80.97


...

,Player,team,Stat,Stat Range,Confidence,recentgames
0,Rudy Gobert,MIN,PTS,7 - 12 - 16,65.60,"[12, 11, 16, 20, 12, 15, 13, 10, 19, 16]"
1,Jaden McDaniels,MIN,REB,3 - 6 - 8,65.51,"[8, 8, 4, 6, 10, 3, 7, 10, 8, 10]"
2,Jaden McDaniels,MIN,PTS,11 - 17 - 22,65.18,"[11, 16, 16, 13, 29, 17, 13, 20, 15, 27]"






Parlay 11: total confidence score: 73.77


,Player,team,Stat,Stat Range,Confidence,recentgames
0,Giannis Antetokounmpo,MIL,PTS,20 - 26 - 31,79.31,"[34, 24, 19, 30, 37, 32, 26, 29, 28, 27]"
1,Giannis Antetokounmpo,MIL,REB,7 - 11 - 14,72.36,"[10, 12, 17, 9, 11, 15, 12, 9, 19, 10]"
2,Damian Lillard,MIL,PTS,15 - 22 - 28,69.63,"[25, 22, 15, 22, 26, 34, 23, 28, 19, 22]"






Parlay 12: total confidence score: 62.02


,Player,team,Stat,Stat Range,Confidence,recentgames
0,Damian Lillard,MIL,AST,3 - 5 - 6,63.63,"[8, 10, 11, 4, 4, 5, 4, 6, 4, 3]"
1,Kyle Kuzma,MIL,REB,3 - 5 - 6,61.84,"[8, 4, 4, 6, 8, 4, 10, 8, 9, 6]"
2,Giannis Antetokounmpo,MIL,AST,2 - 4 - 5,60.60,"[7, 9, 7, 3, 4, 4, 10, 9, 7, 6]"
